In [21]:
import keras
from keras import Sequential, regularizers
from keras.callbacks import ModelCheckpoint
import random as r
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import os
from itertools import *

train_dir = 'asl_alphabet_train'
test_dir = 'asl_alphabet_test'
data_names = 'A B C D E F G H I J K L M N O P Q R S T U V W X Y Z space del nothing'
data_dict = dict([(j, i) for i, j in zip(range(len(data_names.split(' '))), data_names.split(' '))])
data_inverse = dict([(i, j) for i, j in zip(range(len(data_names.split(' '))), data_names.split(' '))])

In [4]:
def load_data():
    images = []
    labels = []
    print('Loading Data From: ', end='')
    for fo in os.listdir(train_dir):
        if fo == '.DS_Store': continue
        print(fo, end=' ')
        for img in os.listdir(train_dir + '/' + fo):
            if img == '.DS_Store': continue
            img_data = cv2.resize(cv2.imread(train_dir + '/' + fo + '/' + img), (64, 64))
            if fo in data_dict:
                images.append(img_data)
                labels.append(data_dict[fo])
                    
    print('\nAll data loaded!')
                
    images= np.array(images)
    images = images.astype('float32')/255.0
    
    labels = keras.utils.to_categorical(labels)
    
    X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size = 0.05)
        
    print('%d images loaded for training; shape = %s' % (len(X_train), str(X_train.shape)))
    print('%d images loaded for testing; shape = %s' % (len(X_test), str(X_test.shape)))
    
    return X_train, X_test, Y_train, Y_test

    
X_train, X_test, Y_train, Y_test = load_data()

Loading Data From: R U I N G Z T S A F O H del nothing space M J C D V Q X E B K L Y P W 
All data loaded!
82650 images loaded for training; shape = (82650, 64, 64, 3)
4350 images loaded for testing; shape = (4350, 64, 64, 3)


In [11]:
def make_model():
    m = Sequential()
    m.add(Conv2D(16, kernel_size = [3,3], padding = 'same', activation = 'relu', input_shape = (64,64,3)))
    m.add(Conv2D(32, kernel_size = [3,3], padding = 'same', activation = 'relu'))
    m.add(MaxPool2D(pool_size = [3,3]))
    m.add(Conv2D(32, kernel_size = [3,3], padding = 'same', activation = 'relu'))
    m.add(Conv2D(64, kernel_size = [3,3], padding = 'same', activation = 'relu'))
    m.add(MaxPool2D(pool_size = [3,3]))
    m.add(Conv2D(128, kernel_size = [3,3], padding = 'same', activation = 'relu'))
    m.add(Conv2D(256, kernel_size = [3,3], padding = 'same', activation = 'relu'))
    m.add(MaxPool2D(pool_size = [3,3]))
    m.add(BatchNormalization())
    m.add(Flatten())
    m.add(Dropout(0.5))
    m.add(Dense(512, activation = 'relu', kernel_regularizer = regularizers.l2(0.001)))
    m.add(Dense(29, activation = 'softmax'))
    m.compile(optimizer = 'adam', loss = keras.losses.categorical_crossentropy, metrics = ["accuracy"])
    m.summary()
    return m

model = make_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 64, 64, 16)        448       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 21, 21, 32)        9248      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 7, 7, 128)         73856     
__________

In [126]:
def train_model():
    h = model.fit(X_train, Y_train, batch_size = 64, epochs = 1, validation_split = 0.1)
    return h

h = train_model()

Train on 74385 samples, validate on 8265 samples
Epoch 1/1
74385/74385 [==============================] - 737s 10ms/step - loss: 0.2712 - acc: 0.9645 - val_loss: 0.4081 - val_acc: 0.8992


In [12]:
def load_test_data():
    images = []
    labels = []
    for img in os.listdir(test_dir):
        tmp = cv2.imread(test_dir + '/' + img)
        tmp = cv2.resize(tmp, (64, 64))
        images.append(tmp)
        labels.append(data_dict[img.split('_')[0]])
    images = np.array(images)
    images = images.astype('float32')/255.0
    return images, labels

test_images, test_img_labels = load_test_data()


In [13]:
def give_predictions(test_data):
    predictions_classes = []
    for image in test_data:
        image = image.reshape(1,64,64,3)
        pred = model.predict_classes(image)
        predictions_classes.append(pred[0])
    return predictions_classes

In [18]:
pre = give_predictions(test_images)
for i, j in zip(pre, test_img_labels):
    print('Actual: %s | Predicted %s' % (data_inverse[j], data_inverse[i]))
    pass


Actual: F | Predicted J
Actual: G | Predicted J
Actual: L | Predicted B
Actual: M | Predicted J
Actual: R | Predicted J
Actual: S | Predicted J
Actual: X | Predicted J
Actual: Y | Predicted J
Actual: U | Predicted J
Actual: T | Predicted J
Actual: A | Predicted B
Actual: K | Predicted J
Actual: J | Predicted J
Actual: Z | Predicted B
Actual: nothing | Predicted J
Actual: Q | Predicted B
Actual: P | Predicted J
Actual: space | Predicted J
Actual: O | Predicted B
Actual: N | Predicted J
Actual: E | Predicted J
Actual: D | Predicted J
Actual: H | Predicted J
Actual: I | Predicted D
Actual: B | Predicted E
Actual: C | Predicted B
Actual: V | Predicted D
Actual: W | Predicted J


In [ ]:
# for training for long amounts of time
def train_model_long():
    h = model.fit(X_train, Y_train, epochs = 1, validation_split = 0.05, batch_size=64)
    return h

h = train_model_long()

Train on 78517 samples, validate on 4133 samples
Epoch 1/1
12992/78517 [===>..........................] - ETA: 10:20 - loss: 2.2314 - acc: 0.4074

In [ ]:
h

In [ ]:
""